In [2]:
!pip install pysindy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 4.5 MB/s eta 0:00:00


# Run the basic pysindy example

In [3]:
import numpy as np
import pysindy as ps

In [4]:
t = np.linspace(0, 1, 100)
x = 3 * np.exp(-2 * t)
y = 0.5 * np.exp(t)
X = np.stack((x, y), axis=-1)  # First column is x, second is y

model = ps.SINDy()
model.fit(X, t=t, feature_names=["x", "y"])
model.print()

(x)' = -2.000 x
(y)' = 1.000 y


### Do inference.. it is very slow

In [6]:
%timeit model.predict(np.atleast_2d(np.random.random(2)))

1.89 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Helper functions for making fast sindy model and saving to json

Note: Claude AI wrote most of this for me at my direction.

In [17]:
import sympy as sp
import json

In [33]:
# Build fast lambda functions: mostly written by Claude
def build_lambda_funcs(feature_names, symbols):
    # replace ' ' with multiplication
    for i, feature in enumerate(feature_names):
      feature = feature.replace(' ', '*')
      feature_names[i] = feature

    # Parse feature names into SymPy expressions
    sympy_exprs = []
    for name in feature_names:
          expr = sp.sympify(name.replace('^', '**').replace(' ', '*'))
          sympy_exprs.append(expr)

    # Convert to fast lambda functions
    lambda_funcs = [sp.lambdify(symbols, expr, 'numpy') for expr in sympy_exprs]

    return lambda_funcs

# When saving: mostly written by Claude
def save_sindy_portable(lambda_funcs, coefficients, filename):
    func_strings = []
    for i, func in enumerate(lambda_funcs):
        try:
            source = inspect.getsource(func).strip()
        except:
            import dill
            source = dill.source.getsource(func).strip()
        func_strings.append(source)

    data = {
        'function_strings': func_strings,
        'coefficients': coefficients.tolist() if hasattr(coefficients, 'tolist') else coefficients
    }

    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

# When loading: mostly written by Claude
def load_sindy_portable(filename):
    with open(filename, 'r') as f:
        data = json.load(f)

    # Create comprehensive namespace with all common math functions
    namespace = {
        'np': np,
        'numpy': np,
        'sin': np.sin,
        'cos': np.cos,
        'tan': np.tan,
        'exp': np.exp,
        'log': np.log,
        'sqrt': np.sqrt,
        'abs': np.abs,
        'sinh': np.sinh,
        'cosh': np.cosh,
        'tanh': np.tanh,
        'arcsin': np.arcsin,
        'arccos': np.arccos,
        'arctan': np.arctan,
        'arctan2': np.arctan2,
        'pi': np.pi,
        'e': np.e,
    }

    # Recreate functions
    lambda_funcs = []
    for func_str in data['function_strings']:
        # Use exec for function definitions
        exec(func_str, namespace)

        # Extract the function from namespace
        if func_str.strip().startswith('lambda'):
            # It's a lambda expression
            lambda_funcs.append(eval(func_str, namespace))
        else:
            # It's a def function - extract function name
            func_name = func_str.split('(')[0].replace('def', '').strip()
            lambda_funcs.append(namespace[func_name])

    coefficients = np.array(data['coefficients'])
    return lambda_funcs, coefficients

# Assemble an inference model
class SINDY_INFERENCE_MODEL:
    def __init__(self, lambda_funcs, coefficients):
        self.lambda_funcs = lambda_funcs
        self.coefficients = coefficients

    def predict(self, x, u=None):
        if u is not None:
            cols = np.hstack([np.ravel(x), np.ravel(u)])
        else:
            cols = np.ravel(x)

        results = np.array([func(*cols) for func in self.lambda_funcs])
        v = self.coefficients@np.atleast_2d(results).T
        return v

### Build lambda functions

In [45]:
x, y = sp.symbols('x y')
symbols = [x, y]
lambda_funcs = build_lambda_funcs(model.get_feature_names(), symbols)

### Wrap the lambda functions and run inference

It is acceptably faster.

In [46]:
sindy_inference_model = SINDY_INFERENCE_MODEL(lambda_funcs, model.coefficients())

In [47]:
%timeit sindy_inference_model.predict(np.atleast_2d(np.random.random(2)))

16.8 µs ± 991 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Save the model as json, reload it, and use it.

This makes the model very portable across python versions, platforms, etc.

In [48]:
save_sindy_portable(lambda_funcs, model.coefficients(), 'sindy_model.json')

In [49]:
# remove this from the namespace to make sure we're not cheating
del(lambda_funcs)

In [50]:
lambda_funcs, coefficients = load_sindy_portable('sindy_model.json')

In [51]:
sindy_inference_model = SINDY_INFERENCE_MODEL(lambda_funcs, coefficients)

In [52]:
%timeit sindy_inference_model.predict(np.atleast_2d(np.random.random(2)))

20.4 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
